# RAP

*La première étape est la récupération du texte. Il existe plusieurs manières de récupérer du texte : soit depuis une base de donnée que vous possédez, soit depuis des fichiers XML ou autres que vous possédez, soit en scrapant des pages comme le font les moteurs de recherches, en utilisant une API.*

*Nous ne traiterons pas cette partie, qui est relativement laborieuse et techniquement peu intéressante. Il existe diverses manières de scraper du texte comme à l'aide des librairies scrapy ou beautifulsoup.*

*Dans mon cas, j'ai scrapé la page wikipédia qui propose une liste des rappeurs français. J'ai ensuite récupéré les paroles des différentes chansons de ces rappeurs sur le site Genius, toujours en scrapant. Je ne suis malheureusement pas autorisé à vous fournir ce jeu de données, libre à vous d'effectuer la même démarche. Vous pouvez aussi utiliser un jeu de données présent par défaut dans la librairie NLTK.*

## Scrapping de la liste des rappeurs français sur Wikipedia

Les deux pages https://fr.wikipedia.org/wiki/Catégorie:Rappeur_français et https://fr.wikipedia.org/w/index.php?title=Cat%C3%A9gorie:Rappeur_fran%C3%A7ais&pagefrom=Menelik%0AM%C3%A9n%C3%A9lik+%28rappeur%29#mw-pages proposent 331 entrées.

Il existe également une page https://fr.wikipedia.org/wiki/Liste_de_musiciens_hip-hop qui propose davantage d'entrées.

C'est cette seconde page que nous allons utiliser, ce qui permet d'élargir.

In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# get the HTML content
url = 'https://fr.wikipedia.org/wiki/Liste_de_musiciens_hip-hop'
response = requests.get(url)

# print(response.status_code)

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')

content_divs = soup.find_all('div', {'class': 'colonnes'})

idx_artist_url_tuples = []
for content_div in content_divs:
        h2 = content_div.find_previous('h2')
        h2 = h2.text[:-len('[modifier | modifier le code]')]
        list_items = content_div.find_all('li')
        for li in list_items:
            artist = li.text
            artist_page_url = None
            a = li.find('a')
            if a is not None:
                if a['href'].startswith('/wiki/'):
                    artist_page_url = a['href']
            if '[' in artist:
                artist = artist[:artist.index('[')]
            idx_artist_url_tuples.append((h2, artist, artist_page_url))

print('nb entries:', len(idx_artist_url_tuples))
display(pd.DataFrame(idx_artist_url_tuples, columns=['section', 'artist', 'page']))

nb entries: 1146


,section,artist,page
0,0–9,100s,/wiki/Kossisko
1,0–9,100 Kila,/wiki/100_Kila
2,0–9,12 Gauge,/wiki/12_Gauge
3,0–9,2$ Fabo,None
4,0–9,2 Chainz,/wiki/2_Chainz
...,...,...,...
1141,Z,Zebra Katz,/wiki/Zebra_Katz
1142,Z,Z-Ro,/wiki/Z-Ro
1143,Z,Zico,/wiki/Zico_(rappeur)
1144,Z,Zed-K,None


## Scrapping sur Genius des paroles d'un artiste

Il faut commencer par scrapper les paroles d'un seul son, d'un seul artiste.

Ceci étant fait, il faut récupérer la liste des morceaux d'un artiste, partant de sa page,

pb : la liste est tronquée : il faut cliquer sur Show all songs, qui affiche une pop up, elle-même à paginer.
On peut poursuivre sur Wikipedia, mais il y a fort à parier que les listes de morceaux ne seront pas présentées de manière régulière.

et relier les deux, pour pouvoir récupérer l'intégrale des paroles de tous les morceaux d'un seul artiste.

Ceci étant fait, il faut relier la liste Wikipedia avec ces fonctions de récupération de paroles.

Un jeu de données intermédiaire intéressant pourrait être la base de la SACEM (11,8 M d'oeuvres) : à première vue, leur base de données n'est pas publique.

Une autre voie à explorer, c'est le scraping via la free API de Spotify.

https://www.youtube.com/watch?v=xyrLcTiMbgc ça n'a pas l'air gratuit.

### Récupérer tous les morceaux de DJ Shadow

https://genius.com/artists/Dj-shadow

https://genius.com/Dj-shadow-six-days-lyrics => une seule page pour un seul son


Trouvé un tuto :
https://medium.com/analytics-vidhya/how-to-scrape-song-lyrics-a-gentle-python-tutorial-5b1d4ab351d2

1. récupérer une clé d'API : https://docs.genius.com/#/getting-started-h1

Tenté.. ça ne marche pas du premier coup, et je n'ai pas plus de temps à y consacrer, ..

In [29]:
GENIUS_API_TOKEN = 'cpPIVXItPPNHmuU7OdLsFHF2Z_yIqD0uS_QkOtRSnpnVOSiVAtcXBLbInIv4vABN'

# Make HTTP requests
import requests  # Scrape data from an HTML document
from bs4 import BeautifulSoup# I/O
import os  # Search and manipulate strings
import re

### Get a list of Genius.com URL’s for a specified number of songs for an artist

In [31]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response  # Get Genius.com song url's from artist object


def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs
    
# DEMO
request_song_url('Lana Del Rey', 2)

JSONDecodeError: [Errno Expecting value] <!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Genius</title>
    <style>*{box-sizing:border-box}body{width:100%;margin:0;color:#fff;background:#000;font-family:Helvetica Neue,Arial,Helvetica,Geneva,sans-serif;font-size:20px;font-weight:700}a,a:active,a:focus{color:#3d85c6;text-decoration:none}a:hover{border-bottom:1px dotted}header{background-color:#ffff64;padding:10px;color:#000}header img{height:20px;display:block;margin:0 auto}main{max-width:860px;margin:0 auto}.text{text-align:center}.alert{margin:40px 20px}h1{font-size:7.5rem;text-transform:uppercase;font-weight:700;margin:0}@media screen and (max-width:420px){h1{font-size:3.5rem}}.distractions{margin:3rem auto 0;width:100%;max-width:700px}.iframe-container{width:100%;height:0;padding-top:56.25%;position:relative}.iframe-conta

In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# get the HTML content
url = 'https://genius.com/Dj-shadow-six-days-lyrics'
response = requests.get(url)

# print(response.status_code)

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
print(soup)

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>Genius</title>
<style>*{box-sizing:border-box}body{width:100%;margin:0;color:#fff;background:#000;font-family:Helvetica Neue,Arial,Helvetica,Geneva,sans-serif;font-size:20px;font-weight:700}a,a:active,a:focus{color:#3d85c6;text-decoration:none}a:hover{border-bottom:1px dotted}header{background-color:#ffff64;padding:10px;color:#000}header img{height:20px;display:block;margin:0 auto}main{max-width:860px;margin:0 auto}.text{text-align:center}.alert{margin:40px 20px}h1{font-size:7.5rem;text-transform:uppercase;font-weight:700;margin:0}@media screen and (max-width:420px){h1{font-size:3.5rem}}.distractions{margin:3rem auto 0;width:100%;max-width:700px}.iframe-container{width:100%;height:0;padding-top:56.25%;position:relative}.iframe-container iframe{max-width:100%;max-height:100%;position:absolut